## Gravitino Trino Example

In this example, we will use `Jupyter` and the `Trino Python Client` to experience `Gravitino`.

In [19]:
# install trino python client and pandas
%pip install trino pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 14.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 6.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
from trino.dbapi import connect

# Create a Trino connector client
conn = connect(
    host="trino",
    port=8080,
    user="admin",
    catalog="catalog_hive",
    schema="http",
)

trino_client = conn.cursor()

## Prepare

Creates a schema named `catalog_hive.company` in Hive, with its location set to`hdfs://hive:9000/user/hive/warehouse/company.db` on HDFS.

In [12]:
trino_client.execute("""
CREATE SCHEMA catalog_hive.company
  WITH (location = 'hdfs://hive:9000/user/hive/warehouse/company.db')
""").fetchall()

TrinoUserError: TrinoUserError(type=USER_ERROR, name=SCHEMA_ALREADY_EXISTS, message="line 2:1: Schema 'catalog_hive.company' already exists", query_id=20240521_074722_00030_i27uz)

Displays the SQL command that was used to create the schema `catalog_hive.company`.

In [49]:
trino_client.execute("""
SHOW CREATE SCHEMA catalog_hive.company
""").fetchall()

[["CREATE SCHEMA catalog_hive.company\nWITH (\n   location = 'hdfs://hive:9000/user/hive/warehouse/company.db'\n)"]]

Create `employees` table

In [15]:
# Create Table
trino_client.execute(
"""
CREATE TABLE catalog_hive.company.employees
(
  name varchar,
  salary decimal(10,2)
)
WITH (
  format = 'TEXTFILE'
)
"""
).fetchall()

[]

In [16]:
# Insert data
print(trino_client.execute("INSERT INTO catalog_hive.company.employees (name, salary) VALUES ('Sam Evans', 55000)").fetchall())

[[1]]


## Simple queries

Some simple query testing.

In [56]:
import pandas as pd

# Show employees table contents
df = pd.DataFrame(trino_client.execute("SELECT * FROM catalog_hive.company.employees").fetchall(), columns=['Name', 'Salary'])

# Display the DataFrame
df

,Name,Salary
0,Sam Evans,55000.00


In [57]:
# Execute the queries and convert the results directly to DataFrames
df_g = pd.DataFrame(trino_client.execute("SHOW SCHEMAS from catalog_hive").fetchall(), columns=['Schema'])
df_g

,Schema
0,company
1,default
2,information_schema
3,sales


In [48]:
h = trino_client.execute("DESCRIBE catalog_hive.company.employees").fetchall()
h

[['name', 'varchar', '', ''], ['salary', 'decimal(10,2)', '', '']]

In [46]:
df_i = pd.DataFrame(trino_client.execute("SHOW TABLES from catalog_hive.company").fetchall(), columns=['Tables'])
df_i

,Tables
0,employees


## Cross-catalog queries

In a company, there may be different departments using different data stacks. In this example, the HR department uses Apache Hive to store its data and the sales department uses PostgreSQL. You can run some interesting queries by joining the two departments' data together with Gravitino.

To know which employee has the largest sales amount:

In [52]:
# Cross-catalog queries
cross_catalog = trino_client.execute("""
SELECT given_name, family_name, job_title, sum(total_amount) AS total_sales
FROM catalog_hive.sales.sales as s,
  catalog_postgres.hr.employees AS e
where s.employee_id = e.employee_id
GROUP BY given_name, family_name, job_title
ORDER BY total_sales DESC
LIMIT 1
""").fetchall()

# Convert the result to a DataFrame
df_j = pd.DataFrame(cross_catalog, columns=['Given Name', 'Family Name', 'Job Title', 'Total Sales'])

df_j

,Given Name,Family Name,Job Title,Total Sales
0,Dale,Lindsey,Sales Assistant,1809.87


To know the top customers who bought the most by state:

In [54]:
# Execute the query
k = trino_client.execute("""
SELECT customer_name, location, SUM(total_amount) AS total_spent
FROM catalog_hive.sales.sales AS s,
  catalog_hive.sales.stores AS l,
  catalog_hive.sales.customers AS c
WHERE s.store_id = l.store_id AND s.customer_id = c.customer_id
GROUP BY location, customer_name
ORDER BY location, SUM(total_amount) DESC
""").fetchall()

# Convert the result to a DataFrame
df_k = pd.DataFrame(k, columns=['Customer Name', 'Location', 'Total Spent'])

# Display the DataFrame
df_k

,Customer Name,Location,Total Spent
0,Harriet Best,Kansas,4229.70
1,Lenore Wilder,Kansas,1594.84
2,Raya Mcguire,Kansas,349.92
3,Perry Tyler,Kansas,99.99
4,Mia Hahn,Kansas,94.97
5,Mia Hahn,Nebraska,2479.82
6,Raya Mcguire,Nebraska,1674.46
7,Erasmus Phelps,Nebraska,1553.79
8,Perry Tyler,Nebraska,1379.94
9,Harriet Best,Nebraska,1039.81


To know the employee's average performance rating and total sales:

In [55]:
# Execute the query
l = trino_client.execute("""
SELECT e.employee_id, given_name, family_name, AVG(rating) AS average_rating, SUM(total_amount) AS total_sales
FROM catalog_postgres.hr.employees AS e,
  catalog_postgres.hr.employee_performance AS p,
  catalog_hive.sales.sales AS s
WHERE e.employee_id = p.employee_id AND p.employee_id = s.employee_id
GROUP BY e.employee_id,  given_name, family_name
""").fetchall()

# Convert the result to a DataFrame
df_l = pd.DataFrame(l, columns=['Employee ID', 'Given Name', 'Family Name', 'Average Rating', 'Total Sales'])

# Display the DataFrame
df_l

,Employee ID,Given Name,Family Name,Average Rating,Total Sales
0,10,Chelsea,Wade,5.000000,299.97
1,13,Risa,Barber,7.000000,1779.72
2,18,Carolyn,Bradshaw,7.000000,1029.90
3,19,Xyla,Le,4.000000,271.92
4,21,Carol,Decker,5.666667,2924.16
5,22,Quemby,Lucas,5.000000,897.47
6,28,Reuben,Rojas,4.250000,3799.68
7,29,Maxwell,Patel,7.000000,2759.88
8,41,Dale,Lindsey,7.000000,1809.87
9,42,Maite,Riddle,5.000000,485.43
